In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dt = pd.read_csv('girlsgroups_live_audience.csv')
X = dt.iloc[:, 1:-1].values
y = dt.iloc[:, -1].values

from sklearn.model_selection import train_test_split

#学習用とテスト用に分ける処理
#test_size=0.3➡テスト用は30%
#random_state=42➡分割のランダム性を固定
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(random_state = 0, max_depth=5)
reg.fit(X_train, y_train)
#説明変数 X の特徴から、目的変数 y（観客数）を予測する規則を学習する
#訓練用とテスト用に分割するのを先にやらないと、過学習になる

#fitとpredictは別。
#fit(X, y)▶ 回帰木に「勉強」をさせる
#predict(X_test)▶ 勉強したルールを使って「答えを予測」する

#r2_score➡決定係数(点数(0～1)) 1が最強
#mean_absolute_error➡平均絶対誤差((予測のズレの平均（人・回数などの実数）))
#小さいほど優秀
from sklearn.metrics import r2_score, mean_absolute_error

#さっき覚えた“ルール”を使って、未知のグループの観客数を予測する
y_pred = reg.predict(X_test)

print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

#回帰木の「木の深さ(max_depth)」を変えて性能を比較するためのループ
#深い → データを細かく分ける → 過学習しやすい
#浅い → 大まかにしか分けない → 精度低いことも
for d in [2, 3, 4, 5, 6, 7, None]:
    model = DecisionTreeRegressor(max_depth=d, random_state=0)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    print(d, r2_score(y_test, pred))

#「特徴量の重要度（feature importance）」を取り出すコード
#dt.columns[1:-1]CSVの列名のうち説明変数の名前だけを取り出している
#reg.feature_importances_➡「どの特徴量をどれだけ重要だと判断したか」を0〜1で示す指標。
for name, imp in zip(dt.columns[1:-1], reg.feature_importances_):
    print(name, round(imp,3))
#➡小数点以下3桁までで四捨五入する関数
#iloc は「行番号・列番号」
#columns は 説明変数の名前を取り出す

reg.predict([[3,18000000,1200000,650000]])

"""
今回の回帰木モデルでは、R2 が約 0.08、MAE も大きく、予測性能はほとんど得られなかった。
max_depth を変えても精度が改善せず、モデルがデータの特徴をうまく捉えられていないことが分かった。
特徴量重要度では「YouTube 登録者数」が 0.885 と極端に高く、他の変数の影響がほぼ見られなかった。
このことから、今回選んだデータ（MV再生数・CD売上・活動年数など）は、ライブの観客人数と明確な関係性を持っていない可能性がある。
回帰木の実験を通じて、モデル以前に「適切な特徴量（集客と本当に関係するデータ）を選ぶ重要性」を強く実感した。

"""

R2: 0.0801296167148553
MAE: 170301.0
2 -0.890731402411626
3 -0.9860524769208379
4 0.08325589008611156
5 0.0801296167148553
6 -0.6434405372785825
7 -0.6434405372785825
None -0.6434405372785825
years_active 0.052
Max_MV_views 0.005
Max_cd_sales 0.058
youtuve_subs 0.885


array([226000.])